In [1]:
import requests, zipfile, io
import pandas as pd
import numpy as np
import bokeh.plotting as bpl
from datetime import datetime as dt
from bokeh.io import export_png

from pathlib import Path
from herramientas import arbol

In [2]:
DATOS_BRUTOS = Path("../datos/brutos/")
DATOS_PROCESADOS = Path("../datos/procesados/")
FIGURAS = Path("../reportes/figuras/")

In [3]:
bpl.output_notebook()

Loading BokehJS ...

Adquisición de datos históricos, esta parte solo es necesaria una vez, guarda los archivos a disco

Carga de datos históricos y actuales y conversión a DataFrame, para cada archivo se agrega una columna al dataframe que hace referencia a la fecha de la publicación del archivo, el país va a estar roto por problemas de encoding. En libre office se pueden abrir como utf8 pero python detecta un caracter que no puede decodificar y no abre los archivos. Se están forzando a 'latin1' pero entonces los acentos no son correctos.

Para descargar los datos corre el _script_ `obtener_datos.py` que se encuentra en la carpeta `src/datos`

desde tu terminal
```bash
python src/datos/obtener_datos.py
```

In [4]:
frames = []
for archivo in DATOS_BRUTOS.glob("*COVID19MEXICO.csv"):
    data = pd.read_csv(archivo, encoding="latin1", low_memory = False,)
    data["FECHA_ARCHIVO"] = dt.strptime(archivo.name[:6],"%y%m%d").date()
    frames.append(data)

In [5]:
len(frames)

5

In [6]:
frames[0].head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI,FECHA_ARCHIVO
0,2020-04-19,09e8dc,2,9,15,1,15,15,37.0,2,...,2,2,2,2,1,99,99,97,1,2020-04-13
1,2020-04-19,0efbaf,2,9,28,2,16,28,32.0,1,...,2,2,2,1,1,99,99,97,97,2020-04-13
2,2020-04-19,091a48,1,12,15,2,15,15,31.0,2,...,2,2,2,2,2,99,99,97,2,2020-04-13
3,2020-04-19,13236c,1,4,2,2,21,2,4.0,2,...,1,2,2,99,2,99,MÃ©xico,97,2,2020-04-13
4,2020-04-19,0cdb12,1,12,9,2,99,9,12.0,2,...,2,2,2,2,1,99,MÃ©xico,97,2,2020-04-13


In [7]:
all_data = pd.concat(frames,axis=0,ignore_index=True)

Conversión de fechas a formato de fecha

In [8]:
variables_fechas = [col for col in all_data.columns if "FECHA_" in col]

In [9]:
all_data[variables_fechas]

,FECHA_ACTUALIZACION,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,FECHA_ARCHIVO
0,2020-04-19,2020-04-09,2020-03-28,9999-99-99,2020-04-13
1,2020-04-19,2020-04-06,2020-04-04,9999-99-99,2020-04-13
2,2020-04-19,2020-04-06,2020-04-04,9999-99-99,2020-04-13
3,2020-04-19,2020-04-02,2020-03-26,9999-99-99,2020-04-13
4,2020-04-19,2020-03-28,2020-03-17,9999-99-99,2020-04-13
...,...,...,...,...,...
208523,2020-04-19,2020-03-15,2020-03-15,9999-99-99,2020-04-14
208524,2020-04-19,2020-03-24,2020-03-17,9999-99-99,2020-04-14
208525,2020-04-19,2020-03-31,2020-03-26,9999-99-99,2020-04-14
208526,2020-04-19,2020-04-08,2020-04-07,9999-99-99,2020-04-14


In [10]:
for variable in variables_fechas:
    all_data[variable] = pd.to_datetime(all_data[variable], format = "%Y-%m-%d", errors='coerce')

In [11]:
all_data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI', 'FECHA_ARCHIVO'],
      dtype='object')

In [12]:
arbol(DATOS_PROCESADOS)

+ ../datos/procesados
    + README.md
    + historia_pacientes_27_05_2020.csv
    + historia_pacientes_28_05_2020.csv


In [13]:
pacientes = pd.read_csv(DATOS_PROCESADOS / 'historia_pacientes_28_05_2020.csv')

In [14]:
columna = "FECHA_UCI"
pacientes[(pacientes[columna].notna())&(pacientes["FECHA_INGRESO"]!=pacientes[columna])]

,ID_REGISTRO,FECHA_SINTOMAS,FECHA_APARICION,FECHA_INGRESO,FECHA_DEF,FECHA_POSITIVO,FECHA_NEGATIVO,FECHA_HOSPITALIZACION,FECHA_AMBULATORIO,FECHA_INTUBACION,FECHA_NO_INTUBACION,FECHA_NO_APL_INTUBACION,FECHA_SE_IGN_INTUBACION,FECHA_NEUMONIA,FECHA_NO_NEUMONIA,FECHA_SE_IGN_NEUMONIA,FECHA_UCI,FECHA_NO_UCI,FECHA_NOAPL_UCI,FECHA_SE_IGN_UCI
1723,0123bc,2020-03-27,2020-04-12,2020-04-01,NaN,2020-04-14,2020-04-01,NaN,2020-04-01,2020-04-14,2020-04-01,NaN,NaN,2020-04-01,NaN,NaN,2020-04-14,2020-04-01,NaN,NaN
2302,018085,2020-03-29,2020-04-12,2020-04-05,2020-04-13,2020-04-05,NaN,NaN,2020-04-05,2020-04-14,2020-04-05,NaN,NaN,2020-04-05,NaN,NaN,2020-04-14,2020-04-05,NaN,NaN
2973,01f58e,2020-03-27,2020-04-12,2020-04-03,NaN,2020-04-03,NaN,NaN,2020-04-03,NaN,2020-04-03,NaN,NaN,2020-04-03,NaN,NaN,2020-04-15,2020-04-03,NaN,NaN
6122,0406b5,2020-04-05,2020-04-12,2020-04-07,2020-04-13,2020-04-07,NaN,NaN,2020-04-07,2020-04-14,2020-04-07,NaN,NaN,2020-04-07,NaN,NaN,2020-04-14,2020-04-07,NaN,NaN
6270,042280,2020-04-03,2020-04-12,2020-04-03,2020-04-08,2020-04-03,NaN,NaN,2020-04-03,2020-04-14,2020-04-03,NaN,NaN,2020-04-14,2020-04-03,NaN,2020-04-14,2020-04-03,NaN,NaN
8355,056b7d,2020-03-26,2020-04-12,2020-03-31,2020-04-05,2020-03-31,NaN,NaN,2020-03-31,2020-03-31,NaN,NaN,NaN,2020-03-31,NaN,NaN,2020-04-13,2020-03-31,NaN,NaN
8723,05ab0f,2020-04-06,2020-04-13,2020-04-13,NaN,2020-04-15,NaN,NaN,2020-04-13,2020-04-15,2020-04-13,NaN,NaN,2020-04-13,NaN,NaN,2020-04-15,2020-04-13,NaN,NaN
9537,062dac,2020-03-27,2020-04-12,2020-04-03,NaN,NaN,2020-04-03,NaN,2020-04-03,2020-04-13,2020-04-03,NaN,NaN,2020-04-03,NaN,NaN,2020-04-13,2020-04-03,NaN,NaN
10114,0693ef,2020-03-20,2020-04-12,2020-03-27,NaN,2020-03-27,NaN,2020-03-27,2020-04-14,NaN,2020-04-14,2020-03-27,NaN,2020-03-27,NaN,NaN,2020-04-14,NaN,2020-03-27,NaN
10163,069ad9,2020-03-29,2020-04-12,2020-04-01,2020-04-14,2020-04-01,NaN,NaN,2020-04-01,2020-04-14,2020-04-01,NaN,NaN,2020-04-01,NaN,NaN,2020-04-14,2020-04-01,NaN,NaN


Graficación de la historia de un paciente

In [15]:
caso_tmp = all_data[all_data["ID_REGISTRO"]=="1e7968"][["ID_REGISTRO","RESULTADO","TIPO_PACIENTE","FECHA_ARCHIVO","FECHA_SINTOMAS","FECHA_INGRESO","FECHA_DEF","INTUBADO","NEUMONIA","UCI"]]
caso_tmp

,ID_REGISTRO,RESULTADO,TIPO_PACIENTE,FECHA_ARCHIVO,FECHA_SINTOMAS,FECHA_INGRESO,FECHA_DEF,INTUBADO,NEUMONIA,UCI
30078,1e7968,1,2,2020-04-13,2020-03-28,2020-03-30,NaT,2,2,2
67143,1e7968,1,2,2020-04-12,2020-03-28,2020-03-30,NaT,2,2,2
112068,1e7968,1,2,2020-04-16,2020-03-28,2020-03-30,2020-04-06,1,1,1
157074,1e7968,1,2,2020-04-15,2020-03-28,2020-03-30,2020-04-06,1,1,1
199317,1e7968,1,2,2020-04-14,2020-03-28,2020-03-30,2020-04-06,1,1,1


In [16]:
no_positivo, positivo, pendiente = "#00BFA5","#DD5246","#F8C600"
ambulatorio, hospitalizado, no_esp = "#FF9633", "#E10C0D", "#D8DBDD"
si, no, no_aplica, se_ignora = "#970A03", "#1F7249", "#E0E0E0", "#A6A6A6"

In [17]:
registro = caso_tmp["ID_REGISTRO"].min()
inicio_sint = caso_tmp["FECHA_SINTOMAS"].min()
deceso = caso_tmp["FECHA_DEF"].min()
recuperacion = inicio_sint+pd.Timedelta('14 days')
final = all_data['FECHA_ARCHIVO'].max()

In [18]:
p = bpl.figure(x_axis_type="datetime",x_range=(inicio_sint-(final-inicio_sint)*0.1,final+(final-inicio_sint)*0.4),
               y_range=[" ","UCI","Intubado","Neumonía","Hospitalización","Resultado pruebas","Síntomas y defunción",""],
               plot_width=1000,plot_height=300,
               title="Paciente " + registro,
               toolbar_location=None)

In [19]:
p.line(x=[inicio_sint]*2,y=[["Síntomas y defunción",-0.45],["Síntomas y defunción",0.45]],color="blue",line_width=3,legend_label="Inicio síntomas")
if not pd.isnull(deceso):
    p.line(x=[deceso]*2,y=[["Síntomas y defunción",-0.45],["Síntomas y defunción",0.45]],color="black",line_width=3,legend_label="Defunción")
    p.line(x=[inicio_sint,deceso],y=[["Síntomas y defunción"],["Síntomas y defunción"]],color="black",line_width=0.8)
else:
    deceso = final
    p.line(x=[inicio_sint,final],y=[["Síntomas y defunción"],["Síntomas y defunción"]],color="black",line_width=0.8)
if (recuperacion <= final)&(recuperacion <= deceso):
    p.line(x=[inicio_sint+pd.Timedelta('14 days')]*2,y=[["Síntomas y defunción",-0.45],["Síntomas y defunción",0.45]],line_dash="dotted",color="purple",line_width=3,legend_label="14 días")

In [20]:
for valor,color,label in zip([3,2,1],[pendiente,no_positivo,positivo],["Prueba Pendiente","Prueba No positiva","Prueba Positiva"]):
    data_temporal = caso_tmp[(caso_tmp["RESULTADO"]==valor)&(caso_tmp["FECHA_ARCHIVO"]<=final)]
    if len(data_temporal)>0:
        p.rect(x=data_temporal["FECHA_ARCHIVO"],y=["Resultado pruebas"]*len(data_temporal),height=0.9,width=pd.Timedelta(1, 'd'),color=color,line_color="white",legend_label=label)

In [21]:
for valor,color,label in zip([1,2,99],[ambulatorio, hospitalizado,no_esp],["Ambulatorio","Hospitalizado","No especificado"]):
    data_temporal = caso_tmp[(caso_tmp["TIPO_PACIENTE"]==valor)&(caso_tmp["FECHA_ARCHIVO"]<=final)]
    if len(data_temporal)>0:
        p.rect(x=data_temporal["FECHA_ARCHIVO"],y=["Hospitalización"]*len(data_temporal),height=0.9,width=pd.Timedelta(1, 'd'),color=color,line_color="white",legend_label=label)

In [22]:
for valor,color,label in zip([97,98,99,2,1],[no_aplica, se_ignora, no_esp, no, si],["No aplica","Se ignora","No especificado","No","Si"]):
    data_temporal = caso_tmp[(caso_tmp["INTUBADO"]==valor)&(caso_tmp["FECHA_ARCHIVO"]<=final)]
    if len(data_temporal)>0:
        p.rect(x=data_temporal["FECHA_ARCHIVO"],y=["Intubado"]*len(data_temporal),height=0.9,width=pd.Timedelta(1, 'd'),color=color,line_color="white",legend_label=label)

In [23]:
for valor,color,label in zip([97,98,99,2,1],[no_aplica, se_ignora, no_esp, no, si],["No aplica","Se ignora","No especificado","No","Si"]):
    data_temporal = caso_tmp[(caso_tmp["NEUMONIA"]==valor)&(caso_tmp["FECHA_ARCHIVO"]<=final)]
    if len(data_temporal)>0:
        p.rect(x=data_temporal["FECHA_ARCHIVO"],y=["Neumonía"]*len(data_temporal),height=0.9,width=pd.Timedelta(1, 'd'),color=color,line_color="white",legend_label=label)

In [24]:
for valor,color in zip([97,98,99,2,1],[no_aplica, se_ignora, no_esp, no, si]):
    data_temporal = caso_tmp[(caso_tmp["UCI"]==valor)&(caso_tmp["FECHA_ARCHIVO"]<=final)]
    p.rect(x=data_temporal["FECHA_ARCHIVO"],y=["UCI"]*len(data_temporal),height=0.9,width=pd.Timedelta(1, 'd'),color=color,line_color="white")

In [25]:
bpl.show(p)

In [26]:
export_png(p, filename=FIGURAS.joinpath(f"{registro}_{dt.today().strftime('%d-%m-%y')}.png"))

'/Users/ssanchez/Documents/repos/tmp/covid_plots/reportes/figuras/1e7968_222222.png'